In [ ]:
!pip install peewee

     |████████████████████████████████| 855 kB 8.4 MB/s 
  Created wheel for peewee: filename=peewee-3.14.10-cp37-cp37m-linux_x86_64.whl size=616223 sha256=b8249cefec23fae2fe035e4a84bb2da83b357c0a0135607f31c564b0fb9dfe31
  Stored in directory: /root/.cache/pip/wheels/df/4f/73/3228affbec8e2e73e622306ac47ebba39df2170bd784eec69c
Successfully built peewee


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import peewee

db_features_one = peewee.SqliteDatabase("/content/drive/MyDrive/diss_data/features_one.db")
db_features_two = peewee.SqliteDatabase("/content/drive/MyDrive/diss_data/features_two.db")
db_features_three = peewee.SqliteDatabase("/content/drive/MyDrive/diss_data/features_three.db")


class ExtractedFeaturesOne(peewee.Model):
    user_id = peewee.IntegerField()
    data_split_type = peewee.IntegerField()
    feature_array = peewee.BlobField()

    class Meta:
        database = db_features_one
        legacy_table_names = False


class ExtractedFeaturesTwo(peewee.Model):
    user_id = peewee.IntegerField()
    data_split_type = peewee.IntegerField()
    feature_array = peewee.BlobField()

    class Meta:
        database = db_features_two
        legacy_table_names = False


class ExtractedFeaturesThree(peewee.Model):
    user_id = peewee.IntegerField()
    data_split_type = peewee.IntegerField()
    feature_array = peewee.BlobField()

    class Meta:
        database = db_features_three
        legacy_table_names = False


db_features_one.create_tables(
    [
        ExtractedFeaturesOne,
        ExtractedFeaturesTwo,
        ExtractedFeaturesThree
    ]
)


/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
from tqdm import tqdm, trange
from imblearn.over_sampling import SMOTE
import json
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from imblearn.pipeline import Pipeline 
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
import pickle


In [ ]:
f = open("/content/drive/MyDrive/diss_data/suspicious_network.json")
sus_raw = json.load(f)
sus = [int(sus) for sus in sus_raw]
f.close()


In [ ]:
file = open("/content/drive/MyDrive/diss_data/pipe_network.obj",'rb')
pipe = pickle.load(file)
file.close()


In [ ]:
def get_data(query, X, y):
  for feature in tqdm(query):
    X.append(json.loads(feature.feature_array.decode('utf-8')))
    y.append(feature.user_id)


In [ ]:
X = []
y = []


In [ ]:
query = ExtractedFeaturesOne.select().where(~(ExtractedFeaturesOne.user_id << sus))
get_data(query, X, y)


100%|██████████| 613310/613310 [00:26<00:00, 22730.82it/s]


In [ ]:
query = ExtractedFeaturesTwo.select().where(~(ExtractedFeaturesTwo.user_id << sus))
get_data(query, X, y)


100%|██████████| 610781/610781 [00:29<00:00, 20799.88it/s]


In [ ]:
query = ExtractedFeaturesThree.select().where(~(ExtractedFeaturesThree.user_id << sus))
get_data(query, X, y)


100%|██████████| 616756/616756 [00:30<00:00, 20209.96it/s]


In [ ]:
if any(s in y for s in sus):
    raise Exception("SUS!")


In [ ]:
len(X)


1840847

In [ ]:
predictions_dict = {item:[] for item in set(y)}

for pair in tqdm(zip(X, y)):
    _x, _y = pair
    predictions_dict[_y].append(pipe.predict([_x])[0])


1840847it [09:33, 3209.94it/s]


In [ ]:
from collections import Counter
counts = {key:Counter(value) for key, value in predictions_dict.items()}

In [ ]:
confidences = {item:[] for item in set(y)}

for user, prediction_count in counts.items():
    sorted_ = sorted(prediction_count.items(), key=lambda item: item[1])
    total = sum([item[1] for item in sorted_])
    confidence = sorted_[0][1] / total
    confidences[user] = {"u": sorted_[0][0], "c": confidence}


In [ ]:
confidences

{1: {'c': 0.0033222591362126247, 'u': 7187},
 2: {'c': 0.0029850746268656717, 'u': 2293},
 3: {'c': 0.005494505494505495, 'u': 389},
 4: {'c': 0.0035714285714285713, 'u': 6329},
 5: {'c': 0.00398406374501992, 'u': 50605},
 6: {'c': 0.003303964757709251, 'u': 6334},
 7: {'c': 0.005917159763313609, 'u': 76355},
 8: {'c': 0.010526315789473684, 'u': 1004},
 9: {'c': 0.004545454545454545, 'u': 8678},
 10: {'c': 0.0009643201542912247, 'u': 53444},
 11: {'c': 0.001597444089456869, 'u': 24937},
 12: {'c': 0.003968253968253968, 'u': 36913},
 13: {'c': 0.0027573529411764708, 'u': 6334},
 14: {'c': 0.0019305019305019305, 'u': 4380},
 15: {'c': 0.00625, 'u': 2293},
 16: {'c': 0.006134969325153374, 'u': 8739},
 17: {'c': 0.006134969325153374, 'u': 14776},
 18: {'c': 0.000946969696969697, 'u': 15087},
 19: {'c': 0.004219409282700422, 'u': 14776},
 20: {'c': 0.003067484662576687, 'u': 15087},
 21: {'c': 0.001303780964797914, 'u': 49641},
 22: {'c': 0.0024752475247524753, 'u': 694},
 23: {'c': 0.00757

In [ ]:
for user, pair in confidences.items():
    if pair["c"] >= 0.03:
        print(user, ":" , pair)


11720 : {'u': 199, 'c': 0.058823529411764705}
